In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/description.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data_solution.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

2024-03-04 14:45:17.906725: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-04 14:45:17.906780: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-04 14:45:17.908337: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [4]:
df = pd.read_csv("/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt",sep=':::', names=['ID', 'TITLE', 'GENRE', 'DESCRIPTION'])
display(df.head())
print(df.shape)

/tmp/ipykernel_441/3152433989.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv("/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt",sep=':::', names=['ID', 'TITLE', 'GENRE', 'DESCRIPTION'])


,ID,TITLE,GENRE,DESCRIPTION
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
1,2,Cupid (1997),thriller,A brother and sister with a past incestuous r...
2,3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
3,4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
4,5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...


(54214, 4)


In [5]:
'''df_test=pd.read_csv("/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data.txt", sep=':::',names=['ID','TITLE','GENRE','DESCRIPTIONS'])
display(train_data.head())
print(test_data.shape)'''

'df_test=pd.read_csv("/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data.txt", sep=\':::\',names=[\'ID\',\'TITLE\',\'GENRE\',\'DESCRIPTIONS\'])\ndisplay(train_data.head())\nprint(test_data.shape)'

In [6]:
df[0:1]

,ID,TITLE,GENRE,DESCRIPTION
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...


In [7]:
df

,ID,TITLE,GENRE,DESCRIPTION
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
1,2,Cupid (1997),thriller,A brother and sister with a past incestuous r...
2,3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
3,4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
4,5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...
...,...,...,...,...
54209,54210,"""Bonino"" (1953)",comedy,This short-lived NBC live sitcom centered on ...
54210,54211,Dead Girls Don't Cry (????),horror,The NEXT Generation of EXPLOITATION. The sist...
54211,54212,Ronald Goedemondt: Ze bestaan echt (2008),documentary,"Ze bestaan echt, is a stand-up comedy about g..."
54212,54213,Make Your Own Bed (1944),comedy,Walter and Vivian live in the country and hav...


In [8]:
Genre_df = df.loc[df['GENRE'] == 'horror']
Genre_df

,ID,TITLE,GENRE,DESCRIPTION


In [9]:
# Get the index of the description with the maximum length
max_length_index = df['DESCRIPTION'].str.len().idxmax()

# Get the movie description with the maximum length
longest_description = df.loc[max_length_index, 'DESCRIPTION']

# Print the description and its length
print(f"Movie with the longest description:\n{longest_description}\nLength: {len(longest_description)}")


Movie with the longest description:
 Guy Gabaldon died on August 31, 2006 and the world lost someone very special. During the bloody struggle for Saipan in July 1944, U.S. Marine PFC Guy Gabaldon is indeed officially credited with capturing over 1500 Japanese soldiers and civilians - singlehandedly, a record that is untouchable in the annals of American military history. For over sixty years, Guy talked about his exploits on that island, sharing his experience and using his celebrity to inspire new generations who valued bravery and bravado. However, war experience alone does not make a life, and Guy's didn't stop in 1944. He lived many different lives and most importantly he took it upon himself to help the less fortunate, particularly the wayward teenagers he encountered when he returned to the Mariana Islands in 1980, where he would live for twenty years. Guy Gabaldon grew up in East Los Angeles where he spent more time on the streets than at home. He would get into fights and he wa

In [10]:
average_length = df['DESCRIPTION'].str.len().mean()

print(f"The average length of movie descriptions is: {average_length:.2f}")


The average length of movie descriptions is: 600.45


In [11]:
max_length=800
trunc_type='post'
padding_type='post'
oov_tok = ""

In [12]:
num_classes = df['GENRE'].nunique()
print(f"Number of classes: {num_classes}")

Number of classes: 27


In [13]:
df.isna().sum()

ID             0
TITLE          0
GENRE          0
DESCRIPTION    0
dtype: int64

Data pre processing

#Step 1 converting into lower cases, so that two words with such as Ball and ball will be considered as singel token


In [14]:
import re
def clean_text(text):
    # Remove special characters, punctuation, and symbols
    text = re.sub(r'[^\w\s]', '', text)
    return text

In [15]:
df['DESCRIPTION_1']=df['DESCRIPTION'].apply(clean_text)

In [16]:
df['DESCRIPTION_1']

0         Listening in to a conversation between his do...
1         A brother and sister with a past incestuous r...
2         As the bus empties the students for their fie...
3         To help their unemployed father make ends mee...
4         The films title refers not only to the unreco...
                               ...                        
54209     This shortlived NBC live sitcom centered on B...
54210     The NEXT Generation of EXPLOITATION The siste...
54211     Ze bestaan echt is a standup comedy about gro...
54212     Walter and Vivian live in the country and hav...
54213     On Labor Day Weekend 1935 the most intense hu...
Name: DESCRIPTION_1, Length: 54214, dtype: object

In [17]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [18]:
from nltk.corpus import stopwords

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    new_text = [word for word in text.split() if word not in stop_words]
    return ' '.join(new_text)

Removing html tags if there are any

In [19]:
df['DESCRIPTION_1']=df['DESCRIPTION_1'].apply(remove_stopwords)

In [20]:
df['DESCRIPTION_1']

0        Listening conversation doctor parents 10yearol...
1        A brother sister past incestuous relationship ...
2        As bus empties students field trip Museum Natu...
3        To help unemployed father make ends meet Edith...
4        The films title refers unrecovered bodies grou...
                               ...                        
54209    This shortlived NBC live sitcom centered Bonin...
54210    The NEXT Generation EXPLOITATION The sisters K...
54211    Ze bestaan echt standup comedy growing facing ...
54212    Walter Vivian live country difficult time keep...
54213    On Labor Day Weekend 1935 intense hurricane ev...
Name: DESCRIPTION_1, Length: 54214, dtype: object

In [21]:
import spacy 
nlp=spacy.load('en_core_web_sm')

In [22]:
import re 
def tokenize(text):
    tokens=re.split('\W+',text)
    return tokens
df['DESCRIPTION_1']=df['DESCRIPTION_1'].apply(tokenize)

In [23]:
df['DESCRIPTION_1'].head()

0    [Listening, conversation, doctor, parents, 10y...
1    [A, brother, sister, past, incestuous, relatio...
2    [As, bus, empties, students, field, trip, Muse...
3    [To, help, unemployed, father, make, ends, mee...
4    [The, films, title, refers, unrecovered, bodie...
Name: DESCRIPTION_1, dtype: object

In [24]:
import nltk

from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

In [25]:
def lemmatize_text(text):
    """
    Lemmatizes a list of tokens using the WordNet dictionary.

    Args:
        text (list): A list of tokens (words) to be lemmatized.

    Returns:
        list: A list of lemmatized tokens.
    """

    nltk.download('wordnet')  # Download WordNet if not already available

    lemmatizer = WordNetLemmatizer()
    lemmatized_text = [lemmatizer.lemmatize(word, pos='v') for word in text]  # Default POS is verb
    return lemmatized_text


In [26]:
import nltk
print(nltk.data.path)


['/root/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data']


In [27]:
# Assuming your dataframe is named 'df' and the column with tokens is 'DESCRIPTION_1'
df['DESCRIPTION_2'] = df['DESCRIPTION_1'].apply(lemmatize_text)


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


LookupError: 
**********************************************************************
  Resource 'corpora/wordnet' not found.  Please use the NLTK
  Downloader to obtain the resource:  >>> nltk.download()
  Searched in:
    - '/root/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

In [ ]:
lemmatizer = WordNetLemmatizer()
#df["Description_1"] = df["DESCRIPTION_1"].apply(lambda text: " ".join([lemmatiazer.lemmatize(token) for token in nltk.word_tokenize(text)]))
def lemmatize(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text])

In [ ]:
df['Description_1']=df['DESCRIPTION_1'].apply(lemmatize)

In [ ]:
 # ps=PorterStemmer()
# def steam_words(text):
#     return " ".join([ps.stem(word) for word in text])

In [ ]:
# df['DESCRIPTION_1'].apply(steam_words)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Join the lists into strings
df['DESCRIPTION_1'] = df['DESCRIPTION_1'].apply(lambda x: ' '.join(x))

# Now you can fit and transform the data with TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['DESCRIPTION_1']).toarray()

In [ ]:
print(tfidf)
print(tfidf.get_feature_names_out())

In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split
X=df['DESCRIPTION_1']
y=df['GENRE']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
y_train.head()

In [ ]:
y_train.shape

In [ ]:
X_train.head()

In [ ]:
X_train.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
y_train_label=label_encoder.fit_transform(y_train)
y_test_label=label_encoder.transform(y_test)

In [ ]:
y_train_label.shape

In [ ]:
vectorizer = TfidfVectorizer(max_features=100)  
X_train_vectorized = vectorizer.fit_transform(X_train).toarray()
print(X_train_vectorized)
X_test_vectorized = vectorizer.transform(X_test)

In [ ]:
X_train_vectorized.shape

In [ ]:
max_tokens = df['DESCRIPTION_1'].apply(lambda x: len(word_tokenize(x))).max()
max_tokens

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
# model = MLPClassifier()

# # Train the model
# model.fit(X_train_vectorized, y_train_label)

# # Predict on test data
# y_pred = model.predict(X_test_vectorized)

# # Calculate accuracy
# accuracy = accuracy_score(y_test_label, y_pred)
# print("Accuracy:", accuracy)

Using LSTM network 

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
#X_train_vectorized_dense = X_train_vectorized.toarray()
X_test_vectorized_dense = X_test_vectorized.toarray()

In [ ]:
print(X_train_vectorized.shape, y_train_label.shape)
print(X_test_vectorized.shape, y_test_label.shape)

In [ ]:
X_train_padded = pad_sequences(X_train_vectorized, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_vectorized_dense, maxlen=max_length, padding='post')

In [ ]:
# training_padded = np.array(X_train_padded)
# training_labels = np.array(y_train_label)
# testing_padded = np.array(X_test_padded)
# testing_labels = np.array(y_test_label)

In [ ]:
from tensorflow.keras.layers import Layer
from tensorflow.keras import regularizers
from tensorflow.keras import constraints

# Attention Layer
class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self, W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

In [ ]:
vocab_size=60000
embedding_dim=32

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 128, input_length=max_length),  # Experiment with embedding_dim
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),  # Consider LSTMs for sequential data
    AttentionWithContext(), 
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    #tf.keras.layers.Dropout(0.8),  
    tf.keras.layers.Dense(512, activation='relu'),  # Adjust neuron count as needed
    tf.keras.layers.Dense(27, activation='softmax')  # 27 output neurons + softmax for multiclass
])


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
print(X_train_padded.shape, y_train_label.shape)
print(X_test_padded.shape, y_test_label.shape)

In [ ]:
import numpy as np

# Assuming y_train_label and y_test_label are  1D arrays with integer labels
y_train_label_encoded = np.eye(27)[y_train_label]
y_test_label_encoded = np.eye(27)[y_test_label]

# Now y_train_label_encoded and y_test_label_encoded are  2D arrays with one-hot encoded labels


In [ ]:
print(X_train_padded.shape, y_train_label_encoded.shape)
print(X_test_padded.shape, y_test_label_encoded.shape)

In [ ]:
vectorizer = TfidfVectorizer(max_features=1000) 

# Fit and transform the training and test text data
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Convert sparse matrices to dense arrays
X_train_dense = X_train_vectorized.toarray()
X_test_dense = X_test_vectorized.toarray()

# Define maximum sequence length based on the maximum number of tokens
#max_length = 800  

# Pad sequences to ensure uniform length
X_train_padded = pad_sequences(X_train_dense, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_dense, maxlen=max_length, padding='post')

Attention mechanism

In [ ]:
from keras.utils import to_categorical

# Assuming y_train_label and y_test_label are your labels
y_train_label_one_hot = to_categorical(y_train_label, num_classes=27)
y_test_label_one_hot = to_categorical(y_test_label, num_classes=27)

In [ ]:
y_train_label_one_hot[1]

In [ ]:
y_test_label_one_hot[1]

In [ ]:
# from sklearn.svm import SVC
# model = SVC(kernel='linear', C=1.0)
# model.fit(X_train_padded, y_train_label)

In [ ]:
# predictions = model.predict(X_test_padded)

In [ ]:
# from sklearn.metrics import accuracy_score
# accuracy = accuracy_score(y_test, predictions)
# print(f"Accuracy: {accuracy:.2f}")

In [ ]:
num_epochs = 20
history = model.fit(X_train_padded, y_train_label_encoded,batch_size=128, epochs=num_epochs, validation_data=(X_test_padded, y_test_label_encoded), verbose=2) 